In [8]:
! pip install kaggle

In [9]:
#configure the path of .json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
#Upload API
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 87% 70.0M/80.9M [00:00<00:00, 243MB/s]
100% 80.9M/80.9M [00:00<00:00, 241MB/s]


In [11]:
#extracting the dataset from zip
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("Dataset Extracted")

Dataset Extracted


In [12]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
print(stopwords.words('english')) #stopwords are words that doesn't add any meaning to the context

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
t_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [16]:
t_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [17]:
#adding columns name
column_names = ['target','ID', 'date', 'flag', 'user', 'text']
t_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names= column_names, encoding = 'ISO-8859-1')
t_data.head()

,target,ID,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
#Check fro missing values
t_data.isnull().sum()

target    0
ID        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
#checking distribution of data; if not even distribution(50:50), perform upsampling or downsampling
#0: Negative 1:Positive
t_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
#Convert target "4" to "1"
t_data.replace({ 'target':{4:1} }, inplace= True) #adding inplace=True for permanent changes
t_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [21]:
#Stemming: A process of reducing a word to it's root word eg: leaf, leaves, leafs : leaf (root word)
port_stem = PorterStemmer()

In [22]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) #only alphabetes, rest will be removed using 're'
  stemmed_content = stemmed_content.lower()           #converting to lower case
  stemmed_content = stemmed_content.split()           #spliting the words
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]  #reduce the word to it's root word
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
t_data['stemmed_content'] = t_data['text'].apply(stemming)

In [ ]:
t_data.head()

In [ ]:
#seperating data and label
x = t_data['stemmed_content'].values
y = t_data['target'].values

In [ ]:
print(x)

In [ ]:
print(y)

In [ ]:
#splitting train and test data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2) # test size: 20% test data, 80% train data; stratify makes proportional split


In [ ]:
print(x.shape, x_train.shape, x_test.shape)

In [ ]:
#converting text data to numeric data = use Vectorizer
v = TfidfVectorizer()
x_train = v.fit_transform(x_train)
x_train = v.transform(x_train)

In [ ]:
print(x_train)

In [ ]:
print(x_test)

In [ ]:
#Training the logistics regression model
model = LogisticRegression(max_iter = 1000)
model.fit(x_train, y_train)

In [ ]:
#Model evaluation on train data : Use accuracy score
x_train_predection = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_predection)

In [ ]:
print('accuracy on training data: ', training_data_accuracy )

In [ ]:
#Model evaluation on test data : Use accuracy score
x_test_predection = model.predict(x_test)
testing_data_accuracy = accuracy_score(y_test, x_test_predection)

In [ ]:
print('accuracy on testing data: ', testing_data_accuracy )

In [ ]:
#To improve accuracy we can use hyper parameter tuning

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb' )) #wb = write only mode in binary

In [ ]:
#loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav' , rb))

In [ ]:
x_new = x_test[200]
print(y_test[200])

prediction = loaded_model.predict(x_new)
print(prediction)

if(prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')